# Opinion Mining and Sentiment Classification of Yelp Reviews

### 1. Data Preprocessing

In [1]:
import re
import nltk
import pandas as pd
import numpy as np

###### 1.1 Data Cleaning

In [2]:
file = open("C:/Users/JoyWa/Desktop/CSC594/Project/YelpReview/Yelp.csv",'r').readlines()

star = []
text = []
tokens = []
taggeds = []

def remove_contraction(document):
    document = document.replace("'m", " am")
    document = document.replace("can't", "can not")
    document = document.replace("Can't", "Can not")
    document = document.replace("won't", "will not")
    document = document.replace("Won't", "Will not")

    document = document.replace("n't", " not")
    document = document.replace("'ll", " will")
    document = document.replace("'ve", " have")
    document = document.replace("'d", " would")
    document = document.replace("'re", " are")

    document = document.replace("He's", "He is")
    document = document.replace("he's", "he is")
    document = document.replace("She's", "She is")
    document = document.replace("she's", "she is")
    document = document.replace("It's", "It is")
    document = document.replace("it's", "it is")
    document = document.replace("That's", "That is")
    document = document.replace("that's", "that is")
    document = document.replace("Here's", "Here is")
    document = document.replace("here's", "here is")
    document = document.replace("There's", "There is")
    document = document.replace("there's", "there is")
    return document
    
def remove_ex_space(s):
    s = re.sub('\s\s+', ' ', s)
    return s

def remove_ex_punctutation(s):
    s = re.sub(r"\s*\.\.+", '.', s)
    s = re.sub(r"\s*\!\!+", '!', s)
    return s
    

# class RepeatReplacer(object) taken from Python 3 Text Processing with NLTK 3 Cookbook - Jacob Perkins
from nltk.corpus import wordnet
class RepeatReplacer(object):
    def __init__(self):
        self.repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
        self.repl = r'\1\2\3'
    def replace(self, word):
        if wordnet.synsets(word):
            return word
        repl_word = self.repeat_regexp.sub(self.repl, word)
        if repl_word != word:
            return self.replace(repl_word)
        else:
            return repl_word
Repeat_Replacer = RepeatReplacer()

for row in file[0:1000]:
    row = row.strip().split('\t')
    row[1] = row[1].strip()
    '''process some emojis
       :( and ): to Frowny
       :), :D, (: to Smiley
       ;) to Winky'''
    row[1] = row[1].replace(':(','Frowny.')
    row[1] = row[1].replace('):','Frowny.')
    row[1] = row[1].replace(':)','Smiley.')
    row[1] = row[1].replace('(:','Smiley.')
    row[1] = row[1].replace(':D','Smiley.')
    row[1] = row[1].replace(';)','Winky.')
    
    '''replace "&" to "and"'''
    row[1] = row[1].replace('&','and')
    
    row[1] = row[1].replace('\\n','')
    row[1] = row[1].replace('\\','')
    row[1] = row[1].replace('buter','butter')
    row[1] = row[1].replace('cokies','cookies')
    row[1] = row[1].replace('schol','school')
    row[1] = row[1].replace('nodle','noodle')
    
    '''remove extra punctutations'''
    row[1] = remove_ex_punctutation(row[1])
    
    '''remove contraction'''
    row[1] = remove_contraction(row[1])
    
    '''remove extra spaces'''
    row[1] = remove_ex_space(row[1])
    
    '''remove Repeated Characters whose length >2'''
    row[1] = Repeat_Replacer.replace(row[1])
     
    token = nltk.word_tokenize(row[1])
    tagged = nltk.pos_tag(token)
    
    star.append(eval(row[0]))
    text.append(row[1])
    tokens.append(token)
    taggeds.append(tagged)

Create Dataframe

In [3]:
review = pd.DataFrame()
review['star'] = star
review['text'] = text
review['tagged_sentences'] = taggeds

###### 1.2 Create new var 'sentiment': positive 1 // negative 0

Keep only the highly positive(4/5) and highly negative(1/2) reviews. 3 is neutral

In [4]:
review = review[(review['star'] > 3) | (review['star'] < 3)]
review['sentiment'] = review['star'].apply(lambda x : 1 if x > 3 else 0)

In [5]:
pos_review = review[review['sentiment'] ==1]
neg_review = review[review['sentiment'] ==0]

### 2. Extract Opinion Segments and Sentiment Analysis - Simple Sentiment Classification

###### 2.1 Extract Opinion Segments using a Regex pattern parser

pattern 1: ('The', 'DT'), ('chicken', 'NN'), ('and', 'CC'), ('vegetables', 'NNS'), ('were', 'VBD'), ('nice', 'JJ'), ('and', 'CC'), ('fresh', 'JJ')

pattern 2: ('great', 'JJ'), ('service', 'NN'), ('and', 'CC'), ('tasty', 'JJ'), ('fod', 'NN')

In [6]:
grammar = r"""
Nominal:{<NN\w?\w?>?<NN\w?\w?>}
o1: {<DT>?<Nominal><CC>?<DT>?<Nominal>?<VB\w?>+<RB\w?>*<JJ\w?>+<CC>?<JJ\w?>*}
RBJJN: {<RB\w?>*<JJ\w?>+<Nominal>}
o2: {<RBJJN>+(<,>?<CC>?<RBJJN>)*}
"""
cp = nltk.RegexpParser(grammar)

In [7]:
tag_lst = []
for tagged in pos_review['tagged_sentences']:
    tree = cp.parse(tagged)
    for subtree in tree.subtrees():
        if subtree.label() == 'o1' or subtree.label() == 'o2':
            tag_lst.append(subtree.leaves())
            print(subtree.leaves())
            #print(subtree)

[('delicious', 'JJ'), ('yogurt', 'NN'), ('shop', 'NN')]
[('super', 'JJ'), ('fun', 'NN'), ('flavors', 'NNS')]
[('normal', 'JJ'), ('lunch', 'NN')]
[('healthy', 'JJ'), ('peanut', 'NN'), ('buter', 'NN')]
[('perfect', 'JJ'), ('lunch', 'NN')]
[('adorable', 'JJ'), ('chihuahua', 'NN'), ('mix', 'NN')]
[('litle', 'JJ'), ('dog', 'NN')]
[('litle', 'JJ'), ('bit', 'NN')]
[('litle', 'JJ'), ('chihuahua', 'NN'), ('mix', 'NN')]
[('litle', 'JJ'), ('dog', 'NN')]
[('litle', 'JJ'), ('bit', 'NN')]
[('The', 'DT'), ('staf', 'NN'), ('were', 'VBD'), ('friendly', 'JJ')]
[('litle', 'JJ'), ('chihuahua', 'JJ'), ('mix', 'NN')]
[('swetest', 'JJS'), ('litle', 'NN'), ('dog', 'NN')]
[('few', 'JJ'), ('days', 'NNS')]
[('30-35', 'JJ'), ('minutes', 'NNS')]
[('new', 'JJ'), ('tires', 'NNS')]
[('great', 'JJ'), ('service', 'NN')]
[('garlic', 'JJ'), ('french', 'JJ'), ('fries', 'NNS')]
[('great', 'JJ'), ('service', 'NN')]
[('authentic', 'JJ'), ('mexican', 'JJ'), ('fod', 'NN')]
[('taco', 'JJ'), ('bel', 'NN')]
[('The', 'DT'), ('asad

[('actual', 'JJ'), ('human', 'JJ'), ('interaction', 'NN')]
[('nice', 'JJ'), ('touch', 'NN')]
[('nice', 'JJ'), ('folks', 'NNS')]
[('areas', 'NNS'), ('are', 'VBP'), ('fantastic', 'JJ')]
[('smal', 'JJ'), ('sand', 'NN'), ("'beach", 'NNP')]
[('local', 'JJ'), ('friends', 'NNS')]
[('loby', 'JJ'), ('bar-', 'NN')]
[('The', 'DT'), ('loungers', 'NNS'), ('and', 'CC'), ('cabanas', 'NNS'), ('are', 'VBP'), ('apointed', 'VBN'), ('very', 'RB'), ('wel', 'JJ')]
[('fantastic', 'JJ'), ('pilows', 'NNS')]
[('stufed', 'JJ'), ('shirt', 'NN')]
[('corporate', 'JJ'), ('polish', 'NN')]
[('realy', 'JJ'), ('god', 'NN')]
[('Special', 'JJ'), ('thanks', 'NNS')]
[('last', 'JJ'), ('wekend', 'NN')]
[('al', 'JJ'), ('spoiled', 'JJ'), ('beach', 'NN'), ('yupies', 'NNS')]
[('own', 'JJ'), ('Sundevil', 'NNP')]
[('This', 'DT'), ('place', 'NN'), ('was', 'VBD'), ('actualy', 'JJ')]
[('big', 'JJ'), ('old', 'JJ'), ('loud', 'JJ'), ('frat', 'NN'), ('boys', 'NNS')]
[('cute', 'JJ'), ('sorority', 'NN'), ('girls', 'NNS')]
[('The', 'DT'), ('

[('SkySong', 'NNP'), ('is', 'VBZ'), ('fantastic', 'JJ')]
[('a', 'DT'), ('hotel.It', 'NN'), ('is', 'VBZ'), ('usualy', 'JJ')]
[('audio/visual', 'JJ'), ('equipment', 'NN')]
[('own', 'JJ'), ('presentation', 'NN')]
[('super', 'JJ'), ('minor', 'JJ'), ('inconvenience', 'NN')]
[('other', 'JJ'), ('organizations', 'NNS')]
[('fod', 'JJ'), ('spread', 'NN')]
[('much', 'JJ'), ('guarante', 'NN')]
[('very', 'RB'), ('pleasant', 'JJ'), ('experience', 'NN')]
[('typicaly', 'JJ'), ('Sheraton', 'NNP')]
[('laptop', 'JJ'), ('video', 'NN')]
[('formal', 'JJ'), ('restaurant', 'NN')]
[('diner/entries', 'NNS'), ('ofering', 'VBG'), ('is', 'VBZ'), ('more', 'RBR'), ('funky', 'JJ')]
[('local', 'JJ'), ('restaurants', 'NNS')]
[('A-Loft', 'JJ'), ('note', 'NN')]
[('formal', 'JJ'), ('resturant', 'NN')]
[('hiden', 'JJ'), ('litle', 'NN'), ('place.I', 'NN')]
[('lunch', 'NN'), ('and', 'CC'), ('the', 'DT'), ('options', 'NNS'), ('were', 'VBD'), ('quite', 'RB'), ('unique', 'JJ')]
[('miso', 'JJ'), ('base', 'NN'), ('ramen', 'NNS')]

[('enjoyable', 'JJ'), ('experience', 'NN')]
[('server', 'NN'), ('was', 'VBD'), ('energetic', 'JJ')]
[('last', 'JJ'), ('time', 'NN')]
[('litle', 'JJ'), ('encouragement', 'NN')]
[('bater', 'NN'), ('and', 'CC'), ('was', 'VBD'), ('very', 'RB'), ('god', 'JJ')]
[('thre', 'JJ'), ('delicious', 'JJ'), ('light', 'NN'), ('and', 'CC'), ('flufy', 'JJ'), ('pancakes', 'NNS')]
[('above', 'JJ'), ('average', 'NN'), (',', ','), ('light', 'JJ'), ('flakey', 'NN'), ('crust', 'NN'), (',', ','), ('and', 'CC'), ('delicious', 'JJ'), ('gravy', 'NN')]
[('perfect', 'JJ'), ('location', 'NN')]
[('Amazing', 'JJ'), ('selection', 'NN')]
[('best', 'JJS'), ('music', 'NN'), ('section', 'NN')]
[('The', 'DT'), ('place', 'NN'), ('is', 'VBZ'), ('very', 'RB'), ('comfortable', 'JJ'), ('and', 'CC')]
[('the', 'DT'), ('fod', 'NN'), ('is', 'VBZ'), ('great', 'JJ')]
[('much', 'JJ'), ('stay', 'NN')]
[('col', 'JJ'), ('place', 'NN')]
[('stars.The', 'NN'), ('owner', 'NN'), ('is', 'VBZ'), ('phenomenal', 'JJ')]
[('her.The', 'JJ'), ('items'

[('perfect', 'JJ'), ('medium', 'NN')]
[('Both', 'DT'), ('meals', 'NNS'), ('were', 'VBD'), ('so', 'RB'), ('god', 'JJ')]
[('whole', 'JJ'), ('bowl', 'NN')]
[('great', 'JJ'), ('experience', 'NN')]
[('litle', 'JJ'), ('mishap', 'NN')]
[('tok', 'JJ'), ('care', 'NN')]
[('Mongolian', 'JJ'), ('BQ', 'NNP'), ('craze', 'NN')]
[('great', 'JJ'), ('solution', 'NN')]
[('The', 'DT'), ('location', 'NN'), ('is', 'VBZ'), ('ideal', 'JJ')]
[('easy', 'JJ'), ('aces', 'NNS')]
[('the', 'DT'), ('fod', 'NN'), ('is', 'VBZ'), ('fresh', 'JJ')]
[('The', 'DT'), ('staf', 'NN'), ('is', 'VBZ'), ('courteous', 'JJ'), ('and', 'CC'), ('helpful', 'JJ')]
[('meal', 'NN'), ('is', 'VBZ'), ('delicious', 'JJ')]
[('he/she', 'JJ'), ('neds', 'NNS')]
[('The', 'DT'), ('meat', 'NN'), ('selection', 'NN'), ('is', 'VBZ'), ('great', 'JJ')]
[('much', 'JJ'), ('uncoked', 'JJ'), ('fod', 'NN')]
[('usualy', 'JJ'), ('end', 'NN')]
[('high', 'JJ'), ('suply', 'NN')]
[('newfangled', 'JJ'), ('yogurt', 'NN'), ('places', 'NNS')]
[('the', 'DT'), ('yogurt', 

[('huge', 'JJ'), ('selection', 'NN')]
[('unique', 'JJ'), ('tols', 'NNS')]
[('recreational', 'JJ'), ('hobies', 'NNS')]
[('new', 'JJ'), ('hoby', 'NN')]
[('neighborhod', 'JJ'), ('sushi', 'NN'), ('bar', 'NN')]
[('god', 'JJ'), ('portions', 'NNS'), (',', ','), ('and', 'CC'), ('great', 'JJ'), ('service', 'NN')]
[('single', 'JJ'), ('rol', 'NN')]
[('the', 'DT'), ('sushi', 'NN'), ('chefs', 'NNS'), ('takes', 'VBZ'), ('special', 'JJ')]
[('salmon/yelowtail/tuna', 'JJ'), ('rol', 'NN')]
[('staf', 'NN'), ('is', 'VBZ'), ('also', 'RB'), ('very', 'RB'), ('atentive', 'JJ')]
[('empty', 'JJ'), ('plates', 'NNS')]
[('diferent', 'JJ'), ('courses', 'NNS')]
[('i', 'NN'), ('play', 'VBP'), ('i', 'JJ')]
[('didnt', 'JJ'), ('hapen', 'NN')]
[('im', 'JJ'), ('gona', 'NN'), ('totaly', 'NN')]
[('this', 'DT'), ('place', 'NN'), ('is', 'VBZ'), ('definitely', 'RB'), ('god', 'JJ')]
[('fod', 'JJ'), ('place', 'NN')]
[('hot', 'JJ'), ('dogs', 'NNS')]
[('whole', 'JJ'), ('life', 'NN')]
[('italian/asian', 'JJ'), ('menu', 'NN')]
[('Ev

[('unasuming', 'JJ'), ('place', 'NN')]
[('here', 'RB'), ('numerous', 'JJ'), ('times', 'NNS')]
[('each', 'DT'), ('time', 'NN'), ('the', 'DT'), ('fod', 'NN'), ('is', 'VBZ'), ('consistently', 'RB'), ('delicious', 'JJ')]
[('best', 'JJS'), ('Pad', 'NNP'), ('Thai', 'NNP')]
[('most', 'JJS'), ('god', 'JJ'), ('Asian', 'JJ'), ('restaurants', 'NNS')]
[('last', 'JJ'), ('visit', 'NN')]
[('previous', 'JJ'), ('hole-in-the-wal', 'JJ'), ('lok', 'NN')]
[('high-end', 'JJ'), ('place', 'NN')]
[('god', 'JJ'), ('Thai', 'NNP'), ('fod', 'NN')]
[('Big', 'JJ'), ('Breakfast', 'NN')]
[('smal', 'JJ'), ('size', 'NN')]
[('hot', 'JJ'), ('spot', 'NN'), ('fol', 'NN')]
[('long', 'JJ'), ('wait', 'NN'), ('deter', 'NN')]
[('daily', 'JJ'), ('special', 'NN')]
[('scrambled', 'JJ'), ('egs', 'NNS')]
[('hot', 'JJ'), ('links', 'NNS'), (',', ','), ('white', 'JJ'), ('chese', 'NN')]
[('delicious', 'JJ'), ('home', 'NN'), ('fries', 'NNS')]
[('blustery', 'JJ'), ('sumer', 'NN'), ('day', 'NN')]
[('frozen', 'JJ'), ('yogurt', 'NN')]
[('pret

[('Younger', 'JJR'), ('crowd', 'NN')]
[('god', 'JJ'), ('music', 'NN')]
[('atmosphere.Strong', 'JJ'), ('pours', 'NNS')]
[('short', 'JJ'), ('list', 'NN')]
[('outstanding', 'JJ'), ('dresing', 'NN')]
[('Wowie', 'JJ'), ('Shrimp', 'NNP')]
[('main', 'JJ'), ('courses', 'NNS')]
[('-excelent', 'JJ'), ('dish', 'NN'), ('and', 'CC'), ('huge', 'JJ'), ('portion', 'NN')]
[('litle', 'JJ'), ('crust', 'NN')]
[('other', 'JJ'), ('friend', 'NN')]
[('traditional', 'JJ'), ('scene', 'NN')]
[('many', 'JJ'), ('restaurants', 'NNS')]
[('very', 'RB'), ('god', 'JJ'), ('experience', 'NN')]
[('more', 'JJR'), ('reason', 'NN')]
[('Phoenix', 'NNP'), ('is', 'VBZ'), ('so', 'RB'), ('great', 'JJ')]
[('most', 'JJS'), ('folks', 'NNS')]
[('South', 'NNP'), ('Phoenix', 'NNP'), ('was', 'VBD'), ('al', 'JJ'), ('farming', 'JJ')]
[('own', 'JJ'), ('homes', 'NNS')]
[('large', 'JJ'), ('lots', 'NNS')]
[('last', 'JJ'), ('10-15', 'JJ'), ('years', 'NNS')]
[('huge', 'JJ'), ("Target/Fry's/Ros/Aplebe", 'NNP')]
[('condos', 'JJ'), ('al', 'NN')]
[

[('so', 'RB'), ('many', 'JJ'), ('great', 'JJ'), ('things', 'NNS')]
[('couple', 'JJ'), ('months', 'NNS')]
[('SHAKING', 'JJ'), ('FIST', 'NNP'), ('AT', 'NNP')]
[('blue', 'JJ'), ('chese', 'NNS')]
[('few', 'JJ'), ('les-od', 'NN'), ('options', 'NNS')]
[('yet', 'RB'), ('delicious', 'JJ'), ('ingredients', 'NNS')]
[('very', 'RB'), ('strong', 'JJ'), ('bacon', 'NN'), ('flavor', 'NN')]
[('britle', 'NN'), ('was', 'VBD'), ('very', 'RB'), ('god.Can', 'JJ')]
[('bad', 'JJ'), ('reviews', 'NNS')]
[('god', 'JJ'), ('piza', 'NN')]
[('best', 'JJS'), ('piza', 'NN')]
[('unhealthy', 'JJ'), ('level', 'NN')]
[('more', 'RBR'), ('serious', 'JJ'), ('tuning', 'NN')]
[('several', 'JJ'), ('unsucesful', 'JJ'), ('hours', 'NNS')]
[('last', 'JJ'), ('resort', 'NN')]
[('first', 'JJ'), ('stop', 'NN')]
[('This', 'DT'), ('Steinmart', 'NNP'), ('had', 'VBD'), ('many', 'JJ')]
[('same', 'JJ'), ('shoes', 'NNS')]
[('this', 'DT'), ('review', 'NN'), ('is', 'VBZ'), ('as', 'RB'), ('much', 'JJ')]
[('future', 'JJ'), ('reminder', 'NN')]
[('

In [8]:
tag_lst_neg = []
for tagged in neg_review['tagged_sentences']:
    tree = cp.parse(tagged)
    for subtree in tree.subtrees():
        if subtree.label() == 'o1' or subtree.label() == 'o2':
            tag_lst_neg.append(subtree.leaves())
            print(subtree.leaves())
            #print(subtree)

[('retail', 'JJ'), ('price', 'NN')]
[('many', 'JJ'), ('shoes', 'NNS')]
[('last', 'JJ'), ('time', 'NN')]
[('Nor', 'NNP'), ('Cal', 'NNP'), ('and', 'CC'), ('people', 'NNS'), ('are', 'VBP'), ('crazy', 'JJ')]
[('first', 'JJ'), ('place', 'NN')]
[('fast', 'JJ'), ('fod', 'NN'), ('restaurant', 'NN')]
[('fast', 'JJ'), ('fod', 'NN')]
[('original', 'JJ'), ('burger', 'NN')]
[('last', 'JJ'), ('ring', 'NN')]
[('last', 'JJ'), ('time', 'NN')]
[('fast', 'JJ'), ('fod', 'NN'), ('joint', 'NN')]
[('few', 'JJ'), ('years', 'NNS')]
[('definitely', 'RB'), ('below', 'JJ'), ('average', 'NN')]
[('only', 'JJ'), ('thing', 'NN')]
[('this', 'DT'), ('place', 'NN'), ('is', 'VBZ'), ('downright', 'VBN'), ('awful', 'JJ')]
[('heavy', 'JJ'), ('set', 'NN'), (',', ','), ('tatoed', 'JJ'), ('gentlemen', 'NNS')]
[('whole', 'JJ'), ('thing', 'NN')]
[('particular', 'JJ'), ('Motel', 'NNP')]
[('long', 'JJ'), ('hard', 'JJ'), ('lok', 'NN')]
[('few', 'JJ'), ('weks', 'NN')]
[('early', 'JJ'), ('evening', 'NN')]
[('The', 'DT'), ('place', 'N

[('other', 'JJ'), ('night', 'NN')]
[('same', 'JJ'), ('kitchen', 'NN')]
[('other', 'JJ'), ('guys', 'NNS')]
[('other', 'JJ'), ('guy', 'NN')]
[('The', 'DT'), ('empanada', 'NN'), ('was', 'VBD'), ('incredibly', 'RB'), ('greasy', 'JJ')]
[('ground', 'NN'), ('bef', 'NN'), ('was', 'VBD'), ('tasty.I', 'JJ')]
[('other', 'JJ'), ('items', 'NNS')]
[('most', 'RBS'), ('boring', 'JJ'), ('bland', 'NN'), ('piza', 'NN')]
[('total', 'JJ'), ('mes', 'NNS')]
[('the', 'DT'), ('piza', 'NN'), ('is', 'VBZ'), ('simply', 'RB'), ('amazing', 'JJ')]
[('cars', 'NNS'), ('parked', 'VBD'), ('outside', 'JJ')]
[('col', 'JJ'), ('place', 'NN')]
[('only', 'JJ'), ('thing', 'NN')]
[('Service', 'NNP'), ('was', 'VBD'), ('god', 'JJ'), ('overal', 'JJ'), ('and', 'CC'), ('atentive', 'JJ')]
[('Fod', 'NNP'), ('was', 'VBD'), ('just', 'RB'), ('okay', 'JJ')]
[('rehearsed', 'JJ'), ('dance', 'NN')]
[('Wait', 'NNP'), ('staf', 'NN'), ('was', 'VBD'), ('slow', 'JJ'), ('and', 'CC')]
[('al', 'JJ'), ('prety', 'NN'), ('comon', 'NN')]
[('other', 'JJ'

[('few', 'JJ'), ('minutes', 'NNS')]
[('litle', 'JJ'), ('story', 'NN')]
[('cold', 'JJ'), ('refreshment', 'NN')]
[('Real', 'JJ'), ('Estate', 'NNP')]
[('web', 'JJ'), ('aces', 'NNS')]
[('most', 'RBS'), ('expensive', 'JJ'), ('phone', 'NN')]
[('celular', 'JJ'), ('neds', 'NNS'), ('Frowny', 'NNP')]
[('magical', 'JJ'), ('powers', 'NNS')]
[('al', 'JJ'), ('beter', 'NN')]
[('average', 'JJ'), ('person', 'NN')]
[('last', 'JJ'), ('few', 'JJ'), ('months', 'NNS')]
[('efin', 'JJ'), ('minutes', 'NNS')]
[('best', 'JJS'), ('solution', 'NN')]
[('late', 'JJ'), ('recomendation', 'NN')]
[('loyal', 'JJ'), ('customers', 'NNS')]
[('not', 'RB'), ('nun', 'JJ'), ('heya', 'NN')]
[('The', 'DT'), ('location', 'NN'), ('was', 'VBD'), ('great', 'JJ')]
[('downtown', 'JJ'), ('Phoenix', 'NNP')]
[('waiter', 'NN'), ('was', 'VBD'), ('very', 'RB'), ('friendly', 'JJ')]
[('The', 'DT'), ('chips', 'NNS'), ('were', 'VBD'), ('very', 'RB'), ('god', 'JJ')]
[('few', 'JJ'), ('dors', 'NNS')]
[('comon', 'JJ'), ('knowledge', 'NN')]
[('own', 

###### 2.2	Compute Polarity Score for matched segments

In [9]:
from nltk.corpus import wordnet as wn

def to_wntag(pos):
    """
    Given a Penn tag, returns the corresponding Wordnet pos tag.
    Note that WordNet only contain nouns/verbs/adjectives/adverbs.
    """
    ch = pos[0]
    if ch == 'N':
        return wn.NOUN #noun
    elif ch == 'V':
        return wn.VERB # verb
    elif ch == 'J':
        return wn.ADJ # adjective
    elif ch == 'R':
        return wn.ADV # adverb
    else:
        return None

def wn_lemmatize(token, wntag):
    """
    Applies WordNet morpher (which requires a pos) and 
    returns the morphological lemma form, or None.
    """
    return wn.morphy(token, wntag)

In [10]:
negcontractions = ["n't", "'t"]
negwords = ['not', 'no', "nor"]
stopwords = nltk.corpus.stopwords.words('english')

In [11]:
from nltk.corpus import sentiwordnet as swn
def polarity_score(tag_lst):
    score_lst = []
    for sent in tag_lst:
        score = 0
        segment_score = 0
        for i in sent:
            w = i[0]
            w = w.lower()
            tag = i[1]
            if "n't" in w:
                w = w.split("n't")[0] + " not"
            if "'t" in w:
                w = w.split("'t")[0] + " not"

            if w not in stopwords or w in negwords:
            # a word is a tuple (w, tag) such as ('bathrooms', 'NNS')
                wntag = to_wntag(tag)
                if wntag is not None: # in WordNet, either noun/verb/adj/adv
                    lemma = wn_lemmatize(w, wntag) #lemmatize token
                    if lemma is None:
                        continue

                    synsets = wn.synsets(lemma, pos=wntag) #all synsets of the word
                    if not synsets:
                        continue

                # take the first sense
                    synset = synsets[0]
                    swn_synset = swn.senti_synset(synset.name()) #SentiWordNet synset
                    pscore = swn_synset.pos_score() - swn_synset.neg_score() #polarity score for the word
                    score += pscore
        segment_score = score/len(sent)
        score_lst.append(segment_score)
    return score_lst

In [12]:
score_lst_pos = polarity_score(tag_lst)
print('-------------------------------------------------')
print('The percentage of matched positive reviews = {}%'.format(round(sum([1 for x in score_lst_pos if x > 0])/len(score_lst_pos)*100),2))
score_lst_neg = polarity_score(tag_lst_neg)
print('-------------------------------------------------')
print('The percentage of matched negtive reviews = {}%'.format(round(sum([1 for x in score_lst_neg if x < 0])/len(score_lst_neg)*100),2))

-------------------------------------------------
The percentage of matched positive reviews = 30%
-------------------------------------------------
The percentage of matched negtive reviews = 21%


### 3. Sentiment Classification with Machine Learning Methodologies

###### 3.1 Counting Word Frequency

In [13]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

from collections import Counter

def counting_word_frequency(tagged_sentence):
    counter = Counter()
    for (word,tag) in tagged_sentence:
        word = re.sub("[^a-zA-Z]", "", word)
        word = word.strip().lower()
        if len(word) > 2 and word not in stop_words:
            word = lemmatizer.lemmatize(word)
            counter[word] +=1
    return counter

In [14]:
review['word_counts'] = review['tagged_sentences'].apply(counting_word_frequency)

###### 3.2 Features Engineering: transforming raw data into features

In [15]:
from sklearn.feature_extraction import DictVectorizer

dict_vect = DictVectorizer()
dict_tf = dict_vect.fit_transform(review['word_counts'])
dict_tf_array = dict_tf.toarray()

In [16]:
df = pd.DataFrame(dict_tf_array, columns=dict_vect.get_feature_names())
df.head(5)

,aaron,aback,abandon,abe,abide,ability,abita,able,abodova,abound,...,ziped,ziti,zolicofer,zombie,zone,ztejas,zuca,zuchini,zumba,zuzu
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Split into training set and test set 70% & 30%

###### 3.3	Apply Supervised Learning Classifiers

In [18]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test  = train_test_split(dict_tf_array, review['sentiment'], test_size=0.3, random_state=33, stratify=review['sentiment'])

For each classifier, I perform parameter configuration to select the best parameter. It can help with the problem of over-fitting.

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from sklearn import metrics

1.Logistic Regression

In [20]:
param_grid = {'penalty':['l1','l2'],'C':[0.01,0.1,1,10,100]}
lrclf = GridSearchCV(LogisticRegression(), param_grid,cv=5)
lrclf = lrclf.fit(X_train, y_train)
lrclf.best_params_

{'C': 0.1, 'penalty': 'l2'}

In [22]:
lrclf = LogisticRegression(C= 0.1, penalty='l2')
lrclf = lrclf.fit(X_train, y_train)
y_pred = lrclf.predict(X_test)
print('Logistic Regression-Average accuracry score on the train dataset is:', round(lrclf.score(X_train, y_train),3))
print('Logistic Regression-Average accuracry score on the test dataset is:', round(lrclf.score(X_test, y_test),3))
print('---------------------------------------------------------------------------')
print(metrics.classification_report(y_test, y_pred))

Logistic Regression-Average accuracry score on the train dataset is: 0.998
Logistic Regression-Average accuracry score on the test dataset is: 0.852
---------------------------------------------------------------------------
             precision    recall  f1-score   support

          0       0.86      0.48      0.62        62
          1       0.85      0.97      0.91       188

avg / total       0.85      0.85      0.84       250



2.Decision Tree

In [24]:
param_grid = {'criterion':['gini','entropy'],'max_depth': np.arange(2, 10), 'min_samples_split':np.arange(2,5),'min_samples_leaf':np.arange(2,5)}
treeclf = GridSearchCV(DecisionTreeClassifier(), param_grid,cv=5)
treeclf = treeclf.fit(X_train, y_train)
treeclf.best_params_

{'criterion': 'gini',
 'max_depth': 9,
 'min_samples_leaf': 4,
 'min_samples_split': 3}

In [25]:
treeclf = DecisionTreeClassifier(criterion= 'gini',max_depth= 9,min_samples_leaf= 4, min_samples_split= 3)
treeclf = treeclf.fit(X_train, y_train)
y_pred = treeclf.predict(X_test)
print('Decision Tree-Average accuracry score on the train dataset is:', round(treeclf.score(X_train, y_train),3))
print('Decision Tree-Average accuracry score on the test dataset is:', round(treeclf.score(X_test, y_test),3))
print('---------------------------------------------------------------------------')
print(metrics.classification_report(y_test, y_pred))

Decision Tree-Average accuracry score on the train dataset is: 0.883
Decision Tree-Average accuracry score on the test dataset is: 0.764
---------------------------------------------------------------------------
             precision    recall  f1-score   support

          0       0.52      0.52      0.52        62
          1       0.84      0.85      0.84       188

avg / total       0.76      0.76      0.76       250



3.Random Forest

In [26]:
rfclf = GridSearchCV(RandomForestClassifier(),param_grid,cv=5)
rfclf = rfclf.fit(X_train, y_train)
rfclf.best_params_

{'criterion': 'gini',
 'max_depth': 7,
 'min_samples_leaf': 3,
 'min_samples_split': 4}

In [27]:
rfclf = RandomForestClassifier(criterion='gini', max_depth=7,min_samples_leaf=3,min_samples_split=4)
rfclf = rfclf.fit(X_train, y_train)
y_pred = rfclf.predict(X_test)
print('Random Forest-Average accuracry score on the train dataset is:', round(rfclf.score(X_train, y_train),3))
print('Random Forest-Average accuracry score on the test dataset is:', round(rfclf.score(X_test, y_test),3))
print('---------------------------------------------------------------------------')
print(metrics.classification_report(y_test, y_pred))

Random Forest-Average accuracry score on the train dataset is: 0.772
Random Forest-Average accuracry score on the test dataset is: 0.764
---------------------------------------------------------------------------
             precision    recall  f1-score   support

          0       1.00      0.05      0.09        62
          1       0.76      1.00      0.86       188

avg / total       0.82      0.76      0.67       250



4.KNN

In [28]:
n_neighbors = 5
knnclf = neighbors.KNeighborsClassifier(n_neighbors) 
knnclf = knnclf.fit(X_train, y_train)
y_pred = knnclf.predict(X_test)

In [30]:
print('KNN-Average accuracry score on the train dataset is:', round(knnclf.score(X_train, y_train),3))
print('KNN-Average accuracry score on the test dataset is:', round(knnclf.score(X_test, y_test),3))
print('---------------------------------------------------------------------------')
print(metrics.classification_report(y_test, y_pred))

KNN-Average accuracry score on the train dataset is: 0.852
KNN-Average accuracry score on the test dataset is: 0.72
---------------------------------------------------------------------------
             precision    recall  f1-score   support

          0       0.43      0.39      0.41        62
          1       0.80      0.83      0.82       188

avg / total       0.71      0.72      0.72       250



### 4. Topic Modeling

#Latent Dirichlet allocation (LDA) is a topic model that generates topics based on word frequency from a set of documents. LDA is particularly useful for finding reasonably accurate mixtures of topics within a given document set.
#get help from https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

###### 4.1.Cleaning documents

In [31]:
'''
Tokenizing: converting a document to its atomic elements.
Stopping: removing meaningless words.
Stemming: merging words that are equivalent in meaning.
'''
def tagged_nouns(tagged_sentence):
    nouns = []
    for (word, tag) in tagged_sentence:
        if word not in stop_words and len(word) > 2 and tag.startswith('NN'):
            nouns.append(lemmatizer.lemmatize(word))            
    return nouns

In [32]:
review['nouns'] = review['tagged_sentences'].apply(tagged_nouns)

In [33]:
review[0:5]

,star,text,tagged_sentences,sentiment,word_counts,nouns
0,4,Thank you thank you thank you! I want to thank...,"[(Thank, NNP), (you, PRP), (thank, VBP), (you,...",1,"{'thank': 4, 'want': 1, 'people': 1, 'made': 2...","[Thank, people, place, hapen.you, yogurt, shop..."
1,5,A Humane Society store at the Biltmore? Intere...,"[(A, DT), (Humane, NNP), (Society, NNP), (stor...",1,"{'humane': 2, 'society': 2, 'store': 3, 'biltm...","[Humane, Society, store, Biltmore, Interesting..."
2,1,Do not buy Nike sneakers if you want to return...,"[(Do, NNP), (not, RB), (buy, VB), (Nike, NNP),...",0,"{'buy': 2, 'nike': 2, 'sneaker': 3, 'want': 1,...","[Nike, sneaker, wek, pair, Nike, price, side, ..."
4,5,The tire presure light came on a day or so ago...,"[(The, DT), (tire, NN), (presure, NN), (light,...",1,"{'tire': 5, 'presure': 1, 'light': 1, 'came': ...","[tire, presure, light, day, husband, tire, air..."
5,4,God Piza and great service!The date diner for ...,"[(God, NNP), (Piza, NNP), (and, CC), (great, J...",1,"{'god': 1, 'piza': 4, 'great': 5, 'service': 2...","[God, Piza, service, date, diner, value, Thurs..."


In [34]:
nouns_lst  = review['nouns'].tolist()

###### 4.2.Constructing a document-term matrix

In [36]:
from gensim import corpora
from gensim.models import LdaModel
dictionary = corpora.Dictionary(nouns_lst)                           
corpus = [dictionary.doc2bow(text) for text in nouns_lst]

###### 4.3. Applying the LDA model

In [40]:
model = LdaModel(corpus, id2word=dictionary, num_topics=30)

###### 4.4 Examining the results

In [42]:
# Each line is a topic with individual topic terms and weights
model.show_topics(num_topics=5)

[(13,
  '0.021*"place" + 0.017*"fod" + 0.012*"restaurant" + 0.011*"time" + 0.007*"people" + 0.007*"wel" + 0.006*"service" + 0.006*"friend" + 0.006*"star" + 0.005*"thing"'),
 (26,
  '0.017*"fod" + 0.009*"chicken" + 0.008*"place" + 0.007*"thing" + 0.007*"god" + 0.006*"staf" + 0.006*"way" + 0.005*"rice" + 0.005*"price" + 0.005*"order"'),
 (2,
  '0.015*"place" + 0.013*"time" + 0.007*"burger" + 0.007*"location" + 0.006*"service" + 0.006*"fod" + 0.006*"thing" + 0.006*"side" + 0.005*"price" + 0.005*"airport"'),
 (15,
  '0.026*"place" + 0.015*"time" + 0.012*"fod" + 0.008*"god" + 0.008*"service" + 0.007*"sauce" + 0.007*"order" + 0.007*"restaurant" + 0.006*"people" + 0.006*"thing"'),
 (28,
  '0.016*"place" + 0.009*"time" + 0.007*"fod" + 0.007*"sauce" + 0.006*"night" + 0.006*"house" + 0.005*"wel" + 0.005*"god" + 0.005*"exhibit" + 0.005*"soda"')]

In [43]:
model.show_topic(1, topn=10)

[('time', 0.014544184),
 ('place', 0.013783222),
 ('service', 0.0068973554),
 ('order', 0.006851297),
 ('window', 0.006315692),
 ('year', 0.006091858),
 ('car', 0.0060835225),
 ('vehicle', 0.0055893688),
 ('phone', 0.00551071),
 ('thing', 0.005392492)]

###### 4.5. Dynamic Visualization of Topic Words using pyLDAvis

In [44]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model, corpus, dictionary)

C:\Users\JoyWa\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
15     8.152140        1       1  0.056921 -0.022391
12     8.039553        1       2  0.059438 -0.028201
5      6.225777        1       3  0.038399 -0.026559
13     4.697011        1       4  0.044015 -0.020506
18     4.146597        1       5  0.060749  0.002025
6      4.099727        1       6  0.036037  0.002002
3      4.093513        1       7  0.054866 -0.015043
16     3.883413        1       8  0.045638  0.010590
1      3.479729        1       9 -0.036141 -0.104940
22     3.454196        1      10  0.044178  0.020592
2      3.392616        1      11  0.022175 -0.037984
11     3.309708        1      12  0.021672  0.011477
8      3.177959        1      13  0.003309  0.032778
23     2.971180        1      14 -0.006901 -0.006467
27     2.937878        1      15 -0.039039 -0.025985
9      2.917674        1      16  0.024752  0.026977
10     2.878417        1      17  0.019927 -0.011830
7      2.779179        1      18 -0.093006 -0.093076
19     2.699656        1      19 -0.004344  0.064661
26     2.694769        1      20  0.000664 -0.020094
17     2.575199        1      21 -0.015998  0.070273
29     2.441339        1      22  0.012692  0.000580
28     2.435954        1      23 -0.072712 -0.002617
14     2.310044        1      24  0.003325  0.031378
25     2.263453        1      25 -0.037291 -0.043295
20     1.934866        1      26 -0.041279  0.007442
0      1.616267        1      27  0.017015  0.036179
21     1.578208        1      28 -0.019868  0.061763
24     1.450271        1      29 -0.124102  0.059099
4      1.363701        1      30 -0.075090  0.021174, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
98    Default  419.000000         fod  419.000000  30.0000  30.0000
102   Default   90.000000        piza   90.000000  29.0000  29.0000
14    Default  670.000000       place  670.000000  28.0000  28.0000
590   Default  127.000000       drink  127.000000  27.0000  27.0000
132   Default  145.000000         bar  145.000000  26.0000  26.0000
215   Default   67.000000      burger   67.000000  25.0000  25.0000
432   Default  159.000000        menu  159.000000  24.0000  24.0000
38    Default   79.000000       store   79.000000  23.0000  23.0000
56    Default  427.000000        time  427.000000  22.0000  22.0000
503   Default   97.000000         rom   97.000000  21.0000  21.0000
27    Default   44.000000         dog   44.000000  20.0000  20.0000
78    Default   21.000000        tire   21.000000  19.0000  19.0000
104   Default  214.000000     service  214.000000  18.0000  18.0000
320   Default   54.000000         kid   54.000000  17.0000  17.0000
202   Default  208.000000  restaurant  208.000000  16.0000  16.0000
295   Default  107.000000        hour  107.000000  15.0000  15.0000
623   Default  106.000000    sandwich  106.000000  14.0000  14.0000
252   Default  111.000000     chicken  111.000000  13.0000  13.0000
157   Default  184.000000       thing  184.000000  12.0000  12.0000
70    Default   47.000000     husband   47.000000  11.0000  11.0000
321   Default  132.000000       night  132.000000  10.0000  10.0000
49    Default  149.000000         day  149.000000   9.0000   9.0000
161   Default   48.000000        wine   48.000000   8.0000   8.0000
264   Default   86.000000        meat   86.000000   7.0000   7.0000
1526  Default   19.000000         Red   19.000000   6.0000   6.0000
451   Default  167.000000       table  167.000000   5.0000   5.0000
273   Default  128.000000       sauce  128.000000   4.0000   4.0000
315   Default   14.000000     exhibit   14.000000   3.0000   3.0000
361   Default   73.000000        rice   73.000000   2.0000   2.0000
828   Default   75.000000    location   75.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
78    Topic30    